In [15]:
# Add variable outputs, assign grid
using SpeedyWeather
using CairoMakie

#spectral_grid = SpectralGrid(trunc=31, nlayers=8)
spectral_grid = SpectralGrid(trunc = 31, nlayers=8, Grid=FullGaussianGrid)
output = NetCDFOutput(spectral_grid, path="F:/SpeedyWeatherRuns/eight_layer")

@kwdef struct TempOutput <: SpeedyWeather.AbstractOutputVariable
    name::String = "temp"
    unit::String = "K"
    long_name::String = "temperature"
    dims_xyzt::NTuple{4, Bool} = (true, true, true, true)
end

@kwdef struct SurPresOutput <: SpeedyWeather.AbstractOutputVariable
    name::String = "spres"
    unit::String = "Pa"
    long_name::String = "surface pressure"
    dims_xyzt::NTuple{4, Bool} = (true, true, false, true)
end

@kwdef struct DivOutput <: SpeedyWeather.AbstractOutputVariable
    name::String = "div"
    unit::String = "s^-1"
    long_name::String = "divergence"
    dims_xyzt::NTuple{4, Bool} = (true, true, true, true)
end

@kwdef struct HumidOutput <: SpeedyWeather.AbstractOutputVariable
    name::String = "humid"
    unit::String = ""
    long_name::String = "specific humidity"
    dims_xyzt::NTuple{4, Bool} = (true, true, true, true)
end

@kwdef struct PrecipOutput <: SpeedyWeather.AbstractOutputVariable
    name::String = "precip"
    unit::String = ""
    long_name::String = "precipitation"
    dims_xyzt::NTuple{4, Bool} = (true, true, false, true)
end
#@kwdef struct PresOutput <: SpeedyWeather.AbstractOutputVariable
   # name::String = "log pres"
   # unit::String = "Pa"
   # long_name::String = "pressure"
   # dims_xyzt::NTuple{4, Bool} = (true, true, false, true)
#end


add!(output, TempOutput(), SurPresOutput(), DivOutput(), HumidOutput(), SpeedyWeather.PrecipitationOutput()...)
SpeedyWeather.path(::TempOutput, simulation) =
    simulation.diagnostic_variables.grid.temp_grid
SpeedyWeather.path(::SurPresOutput, simulation) =
    simulation.diagnostic_variables.grid.pres_grid
SpeedyWeather.path(::DivOutput, simulation) =
    simulation.diagnostic_variables.grid.div_grid
SpeedyWeather.path(::HumidOutput, simulation) =
    simulation.diagnostic_variables.grid.humid_grid
#SpeedyWeather.path(::PresOutput, simulation) =
    #simulation.diagnostic_variables.physics.cos_zenith

time_stepping = Leapfrog(spectral_grid, Δt_at_T31=Minute(30))
ocean = ConstantOceanClimatology(spectral_grid)
planet = Earth(spectral_grid, seasonal_cycle = true, equinox = DateTime(2000, 3, 20))
convection = SimplifiedBettsMiller(spectral_grid)
large_scale_condensation = ImplicitCondensation(spectral_grid)
temperature_relaxation = JablonowskiRelaxation(spectral_grid)
model = PrimitiveWetModel(spectral_grid; output, time_stepping, temperature_relaxation, planet, ocean)
orography = EarthOrography(spectral_grid)
initialize!(orography, model) 
simulation = initialize!(model,time=DateTime(2001,1,1))
# Output state vector every 6 hours = 21600 secs
simulation.model.output.output_dt = Second(21600)

21600 seconds

In [21]:
# Saving Orography
using NCDatasets 
using DataStructures: OrderedDict
using SpeedyWeather

oro_ds = NCDataset("orography_T31.nc","c")
defDim(oro_ds,"lon",96)
defDim(oro_ds,"lat",48)

# Define the variables temperature with the attribute units
v = defVar(oro_ds,"orography", Float32,("lon","lat"), attrib = OrderedDict(
    "units" => "m",
    "long_name" => "Earth Orography"
))

# add additional attributes
v.attrib["comments"] = "Orography used in SpeedyWeather reanalysis run at T31, use as constant"

# write a the complete data set
v[:,:] = orography.orography;
print(v)

close(oro_ds)


orography (96 × 48)
  Datatype:    Float32 (Float32)
  Dimensions:  lon × lat
  Attributes:
   units                = m
   long_name            = Earth Orography
   comments             = Orography used in SpeedyWeather reanalysis run at T31, use as constant


closed Dataset